In [7]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, GRU
from keras.optimizers import SGD
import math
from sklearn.metrics import mean_squared_error


In [8]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\DELL 3510\AppData\Local\Programs\Python\Python38\python.exe -m pip install --upgrade pip' command.


In [9]:
pip install gensim

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\DELL 3510\AppData\Local\Programs\Python\Python38\python.exe -m pip install --upgrade pip' command.


In [ ]:
pip install spacy

In [ ]:
import pandas as pd

# Lire le fichier texte en précisant le séparateur (',' ici, à adapter selon ton fichier)
df = pd.read_csv("household_power_consumption.txt")  # Utilise '\t' si c'est une tabulation

# Sauvegarder en CSV
df.to_csv("household_power_consumption.csv", index=False)

print("Conversion terminée : household_power_consumption.csv est créé !")

Conversion terminée : household_power_consumption.csv est créé !


In [ ]:
df = pd.read_csv("household_power_consumption.csv", sep=";")  # Now columns are separated properly
df.head()

C:\Users\DELL 3510\AppData\Local\Temp\ipykernel_22852\3098965311.py:1: DtypeWarning: Columns (2,3,4,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("household_power_consumption.csv", sep=";")  # Now columns are separated properly


,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,16/12/2006,17:24:00,4.216,0.418,234.840,18.400,0.000,1.000,17.0
1,16/12/2006,17:25:00,5.360,0.436,233.630,23.000,0.000,1.000,16.0
2,16/12/2006,17:26:00,5.374,0.498,233.290,23.000,0.000,2.000,17.0
3,16/12/2006,17:27:00,5.388,0.502,233.740,23.000,0.000,1.000,17.0
4,16/12/2006,17:28:00,3.666,0.528,235.680,15.800,0.000,1.000,17.0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075259 entries, 0 to 2075258
Data columns (total 9 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   Date                   object 
 1   Time                   object 
 2   Global_active_power    object 
 3   Global_reactive_power  object 
 4   Voltage                object 
 5   Global_intensity       object 
 6   Sub_metering_1         object 
 7   Sub_metering_2         object 
 8   Sub_metering_3         float64
dtypes: float64(1), object(8)
memory usage: 142.5+ MB


In [ ]:
import pandas as pd

# Charger les données
df['Datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])
df = df.drop(columns=['Date', 'Time'])  # Supprimer les colonnes inutiles

# Conversion des colonnes numériques
cols_to_convert = ['Global_active_power', 'Global_reactive_power', 'Voltage',
                   'Global_intensity', 'Sub_metering_1', 'Sub_metering_2']

df[cols_to_convert] = df[cols_to_convert].apply(pd.to_numeric, errors='coerce')

# Convertir les colonnes en float (sans toucher à "Datetime")
df[cols_to_convert] = df[cols_to_convert].astype(float)

# Créer la colonne cible (valeur du lendemain)
df['Target'] = df['Global_active_power'].shift(-1)

# Supprimer les lignes avec valeurs manquantes
df = df.dropna()

# Vérification
print(df.dtypes)
print(df.head())


NameError: name 'df' is not defined

In [ ]:
# Séparation des caractéristiques (X) et de la cible (y)
X = df.drop(columns=['Target', 'Datetime'])  # Supprimer la cible et la date pour l'entraînement
y = df['Target']

# Normalisation des caractéristiques
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
# Conversion en DataFrame
df_scaled = pd.DataFrame(X_scaled, columns=X.columns)
df_scaled['Target'] = y.values  # Ajouter la cible non normalisée

# Vérifier le résultat
print(df_scaled.head())
print(df_scaled.dtypes)


NameError: name 'df' is not defined

In [ ]:
from sklearn.model_selection import train_test_split

#Divisez d'abord en ensemble d'entraînement + validation et test
X_train_val, X_test, y_train_val, y_test = train_test_split(df_scaled.drop('Target', axis=1), df_scaled['Target'], test_size=0.2, shuffle=False)

# Divisez maintenant l'ensemble d'entraînement + validation en deux sous-ensembles : entraînement et validation
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1, shuffle=False)
# Vérification des dimensions
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")


NameError: name 'df_scaled' is not defined

In [ ]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\DELL 3510\AppData\Local\Programs\Python\Python38\python.exe -m pip install --upgrade pip' command.


In [ ]:
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, data, target):
        self.data = data
        self.target = target

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return torch.tensor(self.data[idx], dtype=torch.float32), torch.tensor(self.target[idx], dtype=torch.float32)

# Utilisation d'un Dataset personnalisé
train_dataset = CustomDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

# Itérer sur les batches
for X_batch, y_batch in train_loader:
    print(X_batch.shape, y_batch.shape)


NameError: name 'X_train' is not defined

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Définition du modèle LSTM
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(LSTMModel, self).__init__()
        
        # Couche LSTM
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        
        # Couche linéaire pour la prédiction
        self.fc = nn.Linear(hidden_size, output_size)
        
        # Initialisation de la couche LSTM
        self.hidden_size = hidden_size
        self.num_layers = num_layers

    def forward(self, x):
        # Initialisation des états cachés et des cellules de mémoire
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        # Passage par la couche LSTM
        out, _ = self.lstm(x, (h0, c0))
        
        # Utiliser la dernière sortie de la séquence pour la prédiction
        out = out[:, -1, :]  # Dernière sortie (dernière valeur de la séquence)
        
        # Passage par la couche linéaire
        out = self.fc(out)
        
        return out

# Paramètres du modèle
input_size = X_train.shape[2]  # Nombre de caractéristiques (features)
hidden_size = 50  # Nombre de neurones cachés dans la couche LSTM
output_size = 1  # La cible à prédire (par exemple, la consommation énergétique)
num_layers = 2  # Nombre de couches LSTM

# Initialisation du modèle
model = LSTMModel(input_size, hidden_size, output_size, num_layers)
print(model)


NameError: name 'X_train' is not defined